In [1]:
from genotype_network.protein import ProteinGN
from collections import Counter
import networkx as nx

In [2]:
gn = ProteinGN()
gn.read_genotype_network('PB2_2010-2015.pkl')

In [3]:
# Basic statistics about the network.
print(len(gn.edges))
print(len(gn.nodes))

1950
3294


In [4]:
# Print the length of each of the nodes.
lengths = Counter()
for n, d in gn.nodes:
    lengths[len(n)] += 1

lengths.most_common(1)

[(759, 3252)]

In [5]:
# Remove nodes not of the most common length.
for n, d in gn.nodes:
    if len(n) != lengths.most_common(1)[0][0]:
        gn.G.remove_node(n)
        
len(gn.nodes)

3252

In [6]:
# Find all nodes with a given polymorphism at a given position.

def nodes_with_polymorphism(pos, letter):
    """
    Returns the nodes that have a given polymorphism letter at a specified position.
    """
    nodes = set()
    for n, d in gn.nodes:
        if n[pos-1] == letter:
            nodes.add(n)
        
    return nodes

polymorph_nodes = nodes_with_polymorphism(627, 'K')
len(polymorph_nodes)

399

In [7]:
sg = gn.G.subgraph(polymorph_nodes)

In [8]:
def polymorphism_nodes_neighbors(pos, letter, matched=False):
    """
    Finds the nodes that have a given polymorphism.
    
    Then, it returns the neighbors of the polymorphism that do not have that
    given polymorphism at that position.
    """
    from collections import defaultdict
    nodes = nodes_with_polymorphism(pos, letter)
    
    if matched:
        neighbors = defaultdict(list)
    else:
        neighbors = set()
    for n in nodes:
        for n2 in gn.G.neighbors(n):
            if n2[pos-1] != letter:
                if matched:
                    neighbors[n].append(n2)
                else:
                    neighbors.add(n2)
            
    return neighbors

polymorph_neighbors = polymorphism_nodes_neighbors(627, 'K', matched=True)
len(polymorph_neighbors)

11

In [9]:
"""
The polymorphisms of interest are recorded as a dictionary below.
"""

polymorphs_of_interest = {63:['I'],
                          158:['G'],
                          199:['S'],
                          256:['G'],
                          271:['A'],
                          360:['Y'],
                          471:['M'],
                          482:['R'],
                          588:['I'],
                          590:['S'],
                          591:['K', 'R'],
                          627:['K'],
                          636:['F'],
                          661:['T'],
                          667:['I'],
                          701:['N'],
                          702:['R'],
                            }

In [10]:
# Count the total number of nodes + neighbors that will have to be made.

all_nodes = set()
for pos, letters in polymorphs_of_interest.items():
    for letter in letters:
        nodes = nodes_with_polymorphism(pos, letter)
        all_nodes = all_nodes.union(nodes)
        
        neighbors = polymorphism_nodes_neighbors(pos, letter)
        all_nodes = all_nodes.union(neighbors)
        
        print(pos, letter, len(nodes), len(neighbors))
len(all_nodes)

256 G 2 0
482 R 10 0
667 I 682 9
199 S 337 0
360 Y 3236 4
588 I 386 19
702 R 385 4
590 S 1490 14
271 A 1441 0
627 K 399 11
661 T 391 9
471 M 20 6
591 K 17 2
591 R 1108 2
636 F 0 0
701 N 113 5
158 G 2 1
63 I 3011 5


3252

In [11]:
# Count the total number of matched ±polymorphism nodes that will have to be made.

allnodes = set()
for pos, letters in polymorphs_of_interest.items():
    for letter in letters:
        neighbors = polymorphism_nodes_neighbors(pos, letter, matched=True)
        allnodes = allnodes.union(set([i for i in neighbors.keys()]))
        for neighbor_list in neighbors.values():
            for neighbor in neighbors:
                allnodes.union(neighbor)
        print(pos, letter, len(neighbors)*2)
#         allnodes.union(set([for i in neighbors.values()]))
        
print(len(allnodes))

256 G 0
482 R 0
667 I 18
199 S 0
360 Y 8
588 I 28
702 R 8
590 S 28
271 A 0
627 K 22
661 T 18
471 M 12
591 K 4
591 R 4
636 F 0
701 N 8
158 G 2
63 I 10
81


In [12]:
subgraphs = [g for g in nx.connected_component_subgraphs(gn.G)]
len(subgraphs)

1479

In [13]:
subgraph_sizes = [len(g) for g in subgraphs]
Counter(subgraph_sizes)

Counter({1: 1304,
         2: 96,
         3: 24,
         4: 15,
         5: 8,
         6: 7,
         7: 5,
         8: 2,
         9: 3,
         11: 1,
         12: 1,
         13: 1,
         18: 2,
         20: 1,
         40: 1,
         50: 1,
         58: 1,
         72: 1,
         76: 1,
         82: 1,
         208: 1,
         365: 1,
         421: 1})

In [14]:
len(gn.edges)

1946